#### This file creates runtime.csv, which records the runtime for each solution in seconds (on each testcase)

In [ ]:
import os
import math
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.interpolate
import copy 
from statistics import mean 
from scipy.stats import norm, stats

In [ ]:
weights = {'small': 0, 'medium': 75, 'large': 25}
categories = {'small': [0, 1, 2, 3, 4, 5], 'medium': [6, 10, 12, 14], 'large': [15]}
marksFor = dict()

for category, ids in categories.items():
    uniform_weight = weights[category] / len(ids)
    for testid in ids:
        marksFor[testid] = uniform_weight

print (marksFor)
home_dir: str = os.getcwd()
s_id, e_id = 0, 16

s = []
s = os.listdir(os.path.join(home_dir, 'sub'))
all_students = [student.split('_')[1] for student in s]

In [ ]:
final_marks = dict()

for category, ids in categories.items():
    uniform_weight = weights[category] / len(ids)
    for testid in ids:
        final_marks[testid] = uniform_weight

test_cases = list(final_marks.keys())
student_df = pd.DataFrame(columns= ['Entry_no'] + test_cases + ['Total Score'])
student_df['Entry_no'] = all_students
student_df.set_index('Entry_no', inplace=True)
student_df = student_df.replace(np.nan, 0)
student_df

In [ ]:
student_marks = {k:dict() for k in test_cases}

for testid in test_cases:
    logs_dir = os.path.join(home_dir, f'logs_{testid}')
    if os.path.exists(logs_dir):
        verdict_file = os.path.join(logs_dir, f'time_test_{testid}.csv')
        df = pd.read_csv(verdict_file)
        df.rename(columns={'Total Run Time (if correct)': 'Runtime(in sec)'}, inplace=True)
        time_col = 'Runtime(in sec)'
        for idx, row in df.iterrows():
            entry_no = row['Entry_number']
            _entry_no = entry_no.replace('.txt', '')
            run_time = float(row[time_col])
            student_marks[testid][_entry_no] = run_time
        
for testid, scores in student_marks.items():
    for eno, s in scores.items():
        student_df.loc[eno, testid] = s

student_df.replace(np.nan, 0)

In [ ]:
ids = categories['medium'] + categories['large']
final_df = student_df.copy()

# For each test cases check the run-time of each student.
for testid in ids:
    times = list(student_df[testid])
    times = list(filter(lambda time: (time != np.nan and time > 0), times))
    times.sort()
    top = mean(times[:5])
    mid = 4 * top
    # For each student update their score, based on their run time.
    print (f' scaled down marks for each test case: {marksFor[testid]}')
    for _, row in final_df.iterrows():
        if (row[testid] is not np.nan) and (row[testid] > 0.0):
            run_time = row[testid]
            score = max(40, 100 - ((max(0, (run_time - top))) / (mid - top)) * 25)
            row[testid] = score
            row[testid] = (row[testid] * marksFor[testid]) / 100
        else:
            row[testid] = 0.0

final_df = final_df.drop(columns=categories['small'])
cols = list(final_df.columns)
cols.remove('Total Score')

final_df['Total Score'] = final_df.loc[:, cols].sum(axis='columns')

final_df.to_csv(os.path.join(home_dir, 'runtime_score.csv')) # Store the marks assigned to each runtime
student_df.to_csv(os.path.join(home_dir,'runtime.csv')) # Actual runtimes

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import gridspec
import pandas as pd 
import os 

runtime_df = pd.read_csv(os.path.join(home_dir, 'runtime.csv'))
runtime_df.set_index('Entry_no', inplace=True)
runtime_df.drop(columns=['Total Score'], inplace=True)
for testid in runtime_df.columns:
    with open(os.path.join(home_dir, f'test{testid}', 'task1_info.txt')) as f: 
        lines = f.readlines()
        print (lines)

    times = list(runtime_df[testid])
    times = list(filter(lambda time: (time != np.nan and time > 0), times))
    times.sort()

    bins = np.arange(min(times) - 20, max(times) + 30, 15)
    plt.xlim([min(times)- 10, max(times) + 10])
    plt.hist(times, bins=bins, histtype='bar', rwidth=0.9, alpha=0.5)
    plt.xlabel('Run time (seconds)')
    plt.ylabel('Number of students')
    plt.title(f'Test case {testid}')
    if int(testid) >= 5: 
        plt.savefig(os.path.join(home_dir, 'plots', f'test{testid}.png'), format='png')
    plt.show()
    print (f'testid = {testid}, {times}')
